In [ ]:
# Ensembles using weighted averaging 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor

In [3]:
X = np.genfromtxt('data/X_train.txt', delimiter=None)
Y = np.genfromtxt('data/Y_train.txt', delimiter=None)
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)

Xtr, Xva, Ytr, Yva = ml.splitData(X, Y)
Xtr, Ytr = ml.shuffleData(Xtr, Ytr)

In [49]:
# using knn
knn = KNeighborsClassifier(metric_params=None, n_jobs=1, n_neighbors=5, p=1, weights='uniform')
knn.fit(Xtr,Ytr) 
KNNPred = np.vstack((np.arange(Xte.shape[0]), knn.predict_proba(Xte)[:,1])).T
KNNPred[:,1] *= 0.25

In [52]:
np.savetxt('Y_submit_ensemble_KNN.txt', KNNPred,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')

In [5]:
# using neutral networks 
nn = ml.nnet.nnetClassify()
nn.init_weights([Xtr.shape[1], 5, len(np.unique(Ytr))], 'random', Xtr, Ytr)
nn.train(Xtr, Ytr, stopTol=1e-8, stepsize=.25, stopIter=50)
NNPred = np.vstack((np.arange(Xte.shape[0]), nn.predictSoft(Xte)[:,1])).T
NNPred[:,1] *= 0.05

it 1 : Jsur = 0.439479950256, J01 = 0.3411125
it 2 : Jsur = 0.437188313354, J01 = 0.3411125
it 4 : Jsur = 0.438995875095, J01 = 0.3411125
it 8 : Jsur = 0.436759761473, J01 = 0.3411125
it 16 : Jsur = 0.432180661111, J01 = 0.325
it 32 : Jsur = 0.431454931455, J01 = 0.328925


In [54]:
np.savetxt('Y_submit_ensemble_NN.txt', NNPred,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')

In [50]:
# using RAINFORESTS :-) 
clf = RandomForestRegressor(n_estimators=100, random_state=0)
learner = clf.fit(X, Y)
RFPred = np.vstack((np.arange(Xte.shape[0]), learner.predict(Xte)[:])).T
RFPred[:,1] *= 0.7

In [55]:
np.savetxt('Y_submit_ensemble_RF.txt', RFPred,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')

In [57]:
temp = KNNPred[:,1] + NNPred[:,1] + RFPred[:,1]

In [61]:
Yte = []
for i, val in enumerate(temp):
    l = [i, val]
    Yte.append(l)

In [63]:
np.savetxt('Y_submit_ensemble_2.txt', Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')